# The Simple Pendulum with Constant Torque

I find it extremely useful to use simulation to get physical intuition about these systems.  Let's make sure we understand how the simple pendulum moves when it is exposed to a constant torque.

For the duration of this notebook, we'll use the equations of motion $$ml^2 \ddot\theta + b\dot\theta + mgl \sin\theta = u_0,$$ where $u_0$ is our constant scalar torque input.

## Notebook Setup 
The following cell will install Drake, checkout the underactuated repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  Colab will ask you to "Reset all runtimes"; say no to save yourself the reinstall.
- On Binder, the machines should already be provisioned by the time you can run this; it should return (almost) instantly.

More details are available [here](http://underactuated.mit.edu/underactuated.html?chapter=drake).

In [ ]:
try:
  import pydrake
  import underactuated
except ImportError:
  !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
  from jupyter_setup import setup_underactuated
  setup_underactuated()

# Setup matplotlib backend (to notebook, if possible, or inline).  
from underactuated.jupyter import setup_matplotlib_backend
plt_is_interactive = setup_matplotlib_backend()


## Basic simulation

The pendulum is a core example in Drake.  We could certainly load it from a .urdf file, but Drake offers a Pendulum implementation that makes it convenient to manipulate the parameters (and visualize the system with different parameters).

In [ ]:
from math import sin
from IPython.display import display, HTML
import matplotlib.pyplot as plt

from pydrake.all import DiagramBuilder, PlanarSceneGraphVisualizer, SceneGraph, Simulator
from pydrake.examples.pendulum import PendulumGeometry, PendulumPlant
from underactuated.jupyter import SliderSystem

builder = DiagramBuilder()
pendulum = builder.AddSystem(PendulumPlant())

# Setup slider input
torque_system = builder.AddSystem(SliderSystem(min=-5., max=5., description="Torque"))
builder.Connect(torque_system.get_output_port(0), pendulum.get_input_port(0))

# Setup visualization
scene_graph = builder.AddSystem(SceneGraph())
PendulumGeometry.AddToBuilder(builder, pendulum.get_state_output_port(), scene_graph)
visualizer = builder.AddSystem(
    PlanarSceneGraphVisualizer(scene_graph, xlim=[-.7, .7], ylim=[-.7, .7], show=plt_is_interactive))
builder.Connect(scene_graph.get_pose_bundle_output_port(),
                visualizer.get_input_port(0))

diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.)
context = simulator.get_mutable_context()

# TODO(russt): Set/print the parameters.  (Need to add python bindings)
# pendulum_context = diagram.GetMutableSubsystemContext(pendulum, context)
# pendulum.get_mutable_parameters(pendulum_context)

# Set the initial conditions
# TODO(russt): Use pendulum.get_mutable_state(pendulum_context) 
context.SetContinuousState([0., .0]) # theta, thetadot

# Simulate
#visualizer.start_recording()
#duration = 3.0 if get_ipython() else 0.1 # sets a shorter duration during testing
#simulator.AdvanceTo(10)
#ani = visualizer.get_recording_as_animation()
#HTML(ani.to_jshtml())    

Use the slider to set your torque, and then run the cell below to simulate.

In [ ]:
# Simulate
#visualizer.start_recording()
#duration = 3.0 if get_ipython() else 0.1 # sets a shorter duration during testing
context.SetTime(0.)
simulator.AdvanceTo(10)
#ani = visualizer.get_recording_as_animation()
#HTML(ani.to_jshtml())    